In [ ]:
from requests import Request, Session
import json
import numpy as np
import pandas as pd

In [ ]:
import time

In [ ]:
url = 'https://cloud.iexapis.com/stable/stock/AAPL/book'
# url = 'https://cloud.iexapis.com/stable/deep/book'
# url = 'https://sandbox.iexapis.com/stable/deep/book'

In [ ]:
parameters = {
#     'token': 'Tsk_83f2c0eee6bb4600b8d8f9936a5769f2',
    'token': 'sk_b8918e562940417ca9b6615a914b75bd'
#     'symbols': "AAPL"
}
headers = {
    'Accepts': 'application/json'
}

In [ ]:
session = Session()
session.headers.update(headers)

# Function

In [ ]:
def print_data(bids, asks, trades):
    print(bids)
    print(asks)
    print(trades)

In [27]:
def crawl_bid_ask():
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    if 'bids' in data and 'asks' in data and 'trades' in data:
        bids = pd.DataFrame(data.get('bids'))
        asks = pd.DataFrame(data.get('asks'))
        trades = pd.DataFrame(data.get('trades'))
    else:
        columns = ['price','size','timestamp']
        bids = pd.DataFrame(columns=columns)
        asks = pd.DataFrame(columns=columns)
        trades = pd.DataFrame(columns=columns)
    if not trades.empty:
        trades = trades[['price','size','timestamp']]
    print(trades.head(3))
    return bids, asks, trades

In [28]:
def crawl_trade():
    response = session.get(url, params=parameters)

# Initial dataframe
run 1 times

In [ ]:
# bids = pd.DataFrame(columns=['price','size','timestamp'])
# asks = pd.DataFrame(columns=['price','size','timestamp'])
trades = pd.DataFrame(columns=['price','size','timestamp'])
# bids.to_csv('bids.csv', index=False)
# asks.to_csv('asks.csv', index=False)
trades.to_csv('trades.csv', index=False)

# Collect data

In [ ]:
# total_bids = pd.read_csv('bids.csv')
# total_asks = pd.read_csv('asks.csv')
total_trades = pd.read_csv('trades.csv')

## New data

In [ ]:
# for i in range(0,10): # change this into infinite loop when ready
#     bids, asks, trades = crawl()
#     total_bids = total_bids.append(bids).drop_duplicates(subset='timestamp')
#     total_asks = total_asks.append(asks).drop_duplicates(subset='timestamp')
#     total_trades = total_trades.append(trades).drop_duplicates(subset='timestamp')
#     time.sleep(20)

In [ ]:
# total_bids.to_csv('bids.csv', index=False)
# total_asks.to_csv('asks.csv', index=False)
# total_trades.to_csv('trades.csv', index=False)

In [ ]:
i = 0
while i >= 0:
    bids, asks, trades = crawl()
#     total_bids = total_bids.append(bids).drop_duplicates(subset='timestamp')
#     total_asks = total_asks.append(asks).drop_duplicates(subset='timestamp')
    cur_time = round(time.time())
    if not bids.empty and asks.empty:
        bids.to_csv('bids/{0}.csv'.format(cur_time), index=False)
        asks.to_csv('asks/{0}.csv'.format(curtime), index=False)
    total_trades = total_trades.append(trades).drop_duplicates(subset='timestamp')
    i = i + 1
    if i % 2 == 0: # mod 100
#         total_bids.to_csv('bids.csv', index=False)
#         total_asks.to_csv('asks.csv', index=False)
        total_trades.to_csv('trades.csv', index=False)
    time.sleep(20)